In [1]:
from langchain.document_loaders import UnstructuredExcelLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import os
import openpyxl
import json
from openai_api_key import OPENAI_API_KEY

In [2]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [3]:
workbook = openpyxl.load_workbook("responses.xlsx")
worksheet = workbook.active

In [4]:
loader = UnstructuredExcelLoader(
    "faculty.xlsx",
    mode="elements",
)
docs = loader.load()

In [5]:
# Create an index using the loaded documents
index_creator = VectorstoreIndexCreator()
docsearch = index_creator.from_loaders([loader])

In [6]:
# Create a question-answering chain using the index
chain = RetrievalQA.from_chain_type(llm=OpenAI(temperature=0), chain_type="stuff", retriever=docsearch.vectorstore.as_retriever(), input_key="question")

In [7]:
def answer_question(field_of_interest):
    query = f"""
    I have given you a list of my professors and their backgrounds. I want you to take your time to understand and remember this information. I will give you an area of interest and you need to tell me which professors have the most relevant experience in that area, so that I can go and speak to them regarding the same. Give me the names of the professors, along with your reasoning as to why they are the ideal people to speak to. Your output should be in a JSON format, with the keys 'name' and 'reasoning'. An example is given below in double backticks: 
    ``
    [
    {{
        "name": "Prof. A",
        "reasoning": ""
    }},
    {{
        "name": "Prof. B",
        "reasoning": ""
    }}
    ]
    ``
    Output only the JSON response, without the backticks.

    My first prompt is the following, enclosed within triple backticks:
    ```
    I am interested in {field_of_interest}.
    ```
    """
    response = chain({"question": query})
    print(response['result'])
    response = json.loads(response['result'])
    for i in response:
        worksheet.append([field_of_interest, i['name'], i['reasoning']])

In [8]:
answer_question("design thinking and innovation")


[
    {
        "name": "Amit Sheth",
        "reasoning": "Specializes in Design Thinking"
    }
]


In [9]:
answer_question("philosophy")


[
    {
        "name": "Aditya Malik",
        "reasoning": "He is trained in Philosophy, Archaeology, Religious Studies, History, and Social Anthropology and has a doctorate and Habilitation (professorial degree) in South Asian Religions and Modern Indian Studies from the University of Heidelberg."
    },
    {
        "name": "Brainerd Prince",
        "reasoning": "He is trained in Philosophy, Archaeology, Religious Studies, History, and Social Anthropology and has a doctorate and Habilitation (professorial degree) in South Asian Religions and Modern Indian Studies from the University of Heidelberg."
    }
]


In [10]:
answer_question("applied mathematics")


[
    {
        "name": "Amrik Sen",
        "reasoning": "He is a Lecturer of Applied Math and Postdoctoral Fellow at Harvard University, USA"
    },
    {
        "name": "Aditya Malik",
        "reasoning": "He is a Faculty at FLAME University, Pune, India"
    }
]


In [11]:
answer_question("IOT systems")


[
    {
        "name": "Amruta R Behra",
        "reasoning": "Prof. Amruta R Behra specializes in IOT systems."
    }
]


In [12]:
answer_question("Embedded systems and machine learning")


[
    {
        "name": "Anupam Sobti",
        "reasoning": "My research interest lies at the intersection of computer vision, machine learning and embedded systems. Smart sensors, assistive devices, wearable health monitoring systems, etc. require robust and latency-sensitive sensing and actuation. Therefore, these applications require innovation across sensing techniques, perception algorithms and energy optimization techniques in software and hardware."
    }
]


In [13]:
answer_question("Thermodynamics")


[
    {
        "name": "Dr. Shashikant Pawar",
        "reasoning": "Specializes in Fluid Mechanics and Heat Transfer, with research interests in the areas of Environmental Flows, Microclimate Studies, Bio-inspired Energy-efficient and Sustainable Systems, Light Propagation through Turbulent Media, and Engineering Education."
    },
    {
        "name": "Dhiraj Sinha",
        "reasoning": "Specializes in Thermodynamics and electromechanical systems."
    }
]


In [14]:
answer_question("cyber security")


[
    {
        "name": "Gopi",
        "reasoning": "Gopi has experience in operating systems and cybersecurity"
    }
]


In [15]:
answer_question("micro and macro economics")


[
    {
        "name": "Dr. Shashikant Pawar",
        "reasoning": "Dr. Shashikant Pawar has expertise in working with major Indian labor-force and employment surveys, industrial surveys, and judicial data - including PLFS, NSS Employment and Unemployment Surveys, ASI, and NJDG datasets. He has also developed a dynamic heterogeneous-agents model for developing countries which links financial development to per capita income."
    },
    {
        "name": "Assistant Professor of Economics, Plaksha University",
        "reasoning": "Assistant Professor of Economics, Plaksha University has a Ph.D. in Economics from Erasmus University Rotterdam and a M.A. in Economics from Delhi School of Economics. They specialize in Macrofinance, Financial Disclosure and Reporting, Labor Economics, Education, Development Economics, Quantitative Marketing, and Strategic Management."
    }
]


In [16]:
answer_question("biology, preferably genetics")


[
    {
        "name": "Dr. Shashikant Pawar",
        "reasoning": "Dr. Shashikant Pawar is a Mechanical Engineer by training and profession, but his research interests lie in the areas of Environmental Flows, Microclimate Studies, Bio-inspired Energy-efficient and Sustainable Systems, Light Propagation through Turbulent Media, and Engineering Education, which are all related to biology."
    },
    {
        "name": "Manoj Kannan",
        "reasoning": "Manoj Kannan is a professor specializing in Genetics, making him the ideal person to speak to regarding biology, particularly genetics."
    }
]


In [17]:
workbook.save("responses.xlsx")